<a href="https://colab.research.google.com/github/mrp17010/cse4709-project/blob/main/temp/mqtt_pub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install paho-mqtt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 1.5 MB/s eta 0:00:00


In [6]:
import sys
import paho.mqtt.client as paho

client = paho.Client(mqtt.CallbackAPIVersion.VERSION2)

if client.connect("localhost", 1883, 60) != 0:
    print("Couldn't connect to the MQTT broker")
    sys.exit(1)

client.publish("test_topic", "Hi, Paho MQTT client works fine!", 0)
client.disconnect()

NameError: name 'mqtt' is not defined

In [10]:
import paho.mqtt.client as mqtt

def on_subscribe(client, userdata, mid, reason_code_list, properties):
    # Since we subscribed only for a single channel, reason_code_list contains
    # a single entry
    if reason_code_list[0].is_failure:
        print(f"Broker rejected you subscription: {reason_code_list[0]}")
    else:
        print(f"Broker granted the following QoS: {reason_code_list[0].value}")

def on_unsubscribe(client, userdata, mid, reason_code_list, properties):
    # Be careful, the reason_code_list is only present in MQTTv5.
    # In MQTTv3 it will always be empty
    if len(reason_code_list) == 0 or not reason_code_list[0].is_failure:
        print("unsubscribe succeeded (if SUBACK is received in MQTTv3 it success)")
    else:
        print(f"Broker replied with failure: {reason_code_list[0]}")
    client.disconnect()

def on_message(client, userdata, message):
    # userdata is the structure we choose to provide, here it's a list()
    userdata.append(message.payload)
    # We only want to process 10 messages
    if len(userdata) >= 10:
        client.unsubscribe("$SYS/#")

def on_connect(client, userdata, flags, reason_code, properties):
    if reason_code.is_failure:
        print(f"Failed to connect: {reason_code}. loop_forever() will retry connection")
    else:
        # we should always subscribe from on_connect callback to be sure
        # our subscribed is persisted across reconnections.
        client.subscribe("$SYS/#")

mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
mqttc.on_connect = on_connect
mqttc.on_message = on_message
mqttc.on_subscribe = on_subscribe
mqttc.on_unsubscribe = on_unsubscribe

mqttc.user_data_set([])
mqttc.connect("mqtt.eclipseprojects.io")
mqttc.loop_forever()
print(f"Received the following message: {mqttc.user_data_get()}")

Broker granted the following QoS: 0
unsubscribe succeeded (if SUBACK is received in MQTTv3 it success)
Received the following message: [b'mosquitto version 2.0.18', b'3888 seconds', b'1750', b'-3', b'-3', b'1753', b'1753', b'2', b'1787', b'131715.07', b'136774.72', b'157407.08', b'519728.67', b'526285.14', b'605161.98', b'2.26', b'2435.85', b'83686.80', b'117844.85', b'123257.39', b'144114.11', b'504714.33', b'511620.99', b'590740.26', b'7542179.40', b'7789030.37', b'8924947.24', b'31713589.14', b'31763759.21', b'35956593.38', b'1452.51', b'1439.98', b'1395.60', b'1378.01', b'1366.29', b'1324.18', b'43459', b'13362024', b'56105169', b'43459', b'1352510', b'41589', b'0', b'43459', b'1038181744', b'1038229064', b'37706807', b'12444617', b'55115525', b'277412293', b'1486352363', b'834287578', b'3545505267', b'1', b'1', b'1', b'1', b'0', b'1', b'1', b'1', b'1', b'1', b'1', b'1', b'1', b'1']


In [9]:
import time
import paho.mqtt.client as mqtt

def on_publish(client, userdata, mid, reason_code, properties):
    # reason_code and properties will only be present in MQTTv5. It's always unset in MQTTv3
    try:
        userdata.remove(mid)
    except KeyError:
        print("on_publish() is called with a mid not present in unacked_publish")
        print("This is due to an unavoidable race-condition:")
        print("* publish() return the mid of the message sent.")
        print("* mid from publish() is added to unacked_publish by the main thread")
        print("* on_publish() is called by the loop_start thread")
        print("While unlikely (because on_publish() will be called after a network round-trip),")
        print(" this is a race-condition that COULD happen")
        print("")
        print("The best solution to avoid race-condition is using the msg_info from publish()")
        print("We could also try using a list of acknowledged mid rather than removing from pending list,")
        print("but remember that mid could be re-used !")

unacked_publish = set()
mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
mqttc.on_publish = on_publish

mqttc.user_data_set(unacked_publish)
mqttc.connect("mqtt.eclipseprojects.io")
mqttc.loop_start()

# Our application produce some messages
msg_info = mqttc.publish("paho/test/topic", "my message", qos=1)
unacked_publish.add(msg_info.mid)

msg_info2 = mqttc.publish("paho/test/topic", "my message2", qos=1)
unacked_publish.add(msg_info2.mid)

# Wait for all message to be published
while len(unacked_publish):
    time.sleep(0.1)

# Due to race-condition described above, the following way to wait for all publish is safer
msg_info.wait_for_publish()
msg_info2.wait_for_publish()

mqttc.disconnect()
mqttc.loop_stop()

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [11]:
import paho.mqtt.publish as publish

# Publish a single message to a broker
publish.single("paho/test/topic", "payload", hostname="mqtt.eclipseprojects.io")
